# 在PyTorch中构筑卷积神经网络

卷积神经网络是使用卷积层的一组神经网络。在一个成熟的CNN中，往往会涉及到卷积层，池化层，线性层（全连接层）以及各类激活函数。因此，在构筑卷积网络时，需要从整体全部层的需求来进行考虑。

## 1. 二维卷积层nn.Conv2d

![Alt text](image-98.png)

在PyTorch中，卷积层作为构成神经网络的层，自然是nn.Module模块下的类。

按照输入数据的维度，卷积层可以分为三类：处理时序数据的一维卷积，处理图像数据的二维卷积，以及处理视频数据的三维卷积。时序数据是存在时间维度，受时间影响的三维数据，常被用于循环神经网络中，但卷积也可以处理这种数据。视频数据则是由多张图像在时间轴上排列构成的，因此视频数据可以被看作是图像数据的序列。视频数据中的frames是“帧数”，即一个视频中图像的总数量。在之后的课程中，我们会就视频数据及其处理展开详细说明。

按照卷积的操作和效果，又可分为普通卷积，转置卷积，延迟初始化的lazyConv等等。最常用的处理图像的普通卷积nn.Conv2d。其类及其包含的超参数参数内容如下（注意Conv2d是大写）：

In [ ]:
# 查看nn.Conv2d的参数
# nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)

# 参数采用中文注释
# in_channels：输入信号的通道数
# out_channels：卷积产生的通道数
# kernel_size：卷积核的尺寸
# stride：步长
# padding：输入的每一条边补充0的层数
# dilation：卷积核元素之间的间距
# groups：输入通道分组数
# bias：是否添加偏置项


可以看到，除了之前的扫描操作之外，还有许多未知的参数。我们可以通过解析Conv2d的参数来说明卷积操作中的许多细节。需要说明的是，参数groups和dilation分别代表着分组卷积（group convolution）和膨胀卷积（dilated convolution），属于卷积神经网络的入门级操作，但却比我们现在学习的内容更加复杂。之后，我们会详细描述这两种卷积网络的原理与流程，现在，我们还是专注在普通卷积上。

### 1.1 卷积核尺寸kernel_size

kernel_size是我们第一个需要讲解的参数，但同时也是最简单的参数。

![Alt text](image-99.png)

卷积核的高和宽一般用 $K_{H}$，和$K_{W}$表示。在许多其他材料或教材中，如果把卷积核称为filter过滤器，也可能使用字母$F_{H}$和$F_{w}$来表示。卷积核的对整个卷积网络的参数量由很大的影响。在之前的例子中，我们使用3*3结构的卷积核，每个卷积核就会携带9个参数（权重），如果我们使用3*2的结构，每个卷积和就会携带6个参数。

卷积核的尺寸应该如何选择呢？如果你在使用经典架构，那经典架构的论文中所使用的尺寸就是最好的尺寸。如果你在写自己的神经网络，那3*3几乎就是最好的选择。对于这个几乎完全基于经验的问题，我可以提供以下几点提示：

1. 卷积核几乎都是正方形。最初是因为在经典图像分类任务中，许多图像都被处理成正方形或者接近正方形的形状(比如Fashion-MNIST，CIFAR，ImageNet)等等。如果你的原始图像尺寸非常奇妙（例如，非常长或非常宽），你可以使用与原图尺寸比例一致的卷积核尺寸。

2. 卷积核的尺寸最好是奇数，例如3*3,5*5, 7*7等。这是一种行业惯例，传统视觉中认为这是为了让被扫描区域嫩够“中心对称”，无论经历过多少次卷积变换，都能够以正常比例还原图像。

![Alt text](image-100.png)

![Alt text](image-101.png)

相对的，如果扫描区域不是中心对称的，在多次进行卷积操作之后，像素会“偏移”导致图像失真（由最左侧图像变为右侧的状况）。然而，这种说法缺乏有效的理论基础，如果你发现你的神经网络确实更适合偶数卷积核（并且你能够证明，或说明你需要说服的人来接受你的决定），那你可以自由使用偶数卷积核。到目前为止，还没有卷积核的奇偶会对神经网络效果造成影响的明确理论。

![Alt text](image-102.png)

3. 在计算机视觉中，卷积核的尺寸往往都是比较小（相对的，在NLP中，许多网络的卷积核尺寸都可以很大）。这主要是因为较小的卷积核所需要的训练参数会更少，之后我们会详细讨论关于训练参数量的问题。

### 1.2 卷积的输入与输出：in_channels, out_channels, bias

除了Kernel_size之外，还有两个必填参数：in_channels与out_channels。简单来说：in_channels是输入卷积层的图像的通道数或上一层传入特征图的数量，out_channels是指这一层输出的特征图的数量。这两个数量我们都可以自己来确定，但具体扫描流程中的细节都还需要理清。

在之前的例子中，我们在一张图像上使用卷积核进行扫描，得到一张特征图。这里的“被扫描图像”是一个通道，而非一张彩色图片。如果卷积核每每扫描一个通道，就会得到一张特征图，那多通道的图像应该被怎么扫描呢？会有怎么样的输出呢？

在一次扫描中，我们输入了一张拥有三个通道的彩色图像。对于这张图，拥有同样尺寸，但不同具体数值的三个卷积核会分别在三个通道上进行扫描，得出三个相应的“新通道”。由于同一张图片中不同通道的结构一定是一致的，卷积核的尺寸也是一致的，因此卷积操作后得出的“新通道”的尺寸也是一致的。

![Alt text](image-103.png)

得出三个“新通道”后，我们将对应未知的元素相加，形成一张新图，这就是卷积层输入的三彩色图像的第一个特征图。这个操作对于三通道的RGB图像，四通道的RGBA或者CMYK图像都是一致的。只不过，如果是四通道的图像，则会存在4个同样尺寸，但数值不同的卷积核分别扫描4个通道。

![Alt text](image-104.png)

因此，在一次扫描中，无论图像本身有几个通道，卷积核会扫描全部通道之后，将扫描结果加和为一张feature map。所以，一次扫描对应一个feature map，无关原始图像的通道数目是多少，所以out_channels就是扫描次数，这之中卷积核的数量就等与输入的通道数量in_channels x扫描次数out_channels。那对于一个通道，我们还可有可能多次扫描，得出多个feature map吗？当然有可能！卷积核的作用是捕捉特征，同一个通道上很有可能存在多个需要不同的卷积核进行捕捉的特征，例如，能够捕捉到孔雀脖子轮廓的卷积核，就不一定能够捕捉到色彩绚丽的尾巴。因此，对同一个通道提供多个不同的卷积核来进行多次扫描是很普遍的操作。不过，我们并不能够对不同的通道使用不同的卷积核数量。比如，若规定扫描三次，则每次扫描时通道都会分别获得自己的卷积核，我们不能让卷积网络执行类似于“红色通道扫描两次，蓝色通道扫描3次”这样的操作。

需要注意的是，当feature map被输入到下一个卷积层时，它也是被当作“通道”来处理的。不太严谨的说，feature map其实也可以是一种通道，虽然没有定义到具体的颜色，但他其实也是每个元素都在[0, 255]之间的图。这是说，当feature map进入到下一个卷积层时，新卷积层上对所有feature map完成之后，也会将他们的扫描结果加和成一个新feature map。所以，在新卷积层上，依然是一次扫描对应生成一个feature map，无关之前的层上传入的feature map有多少。

---

* 总结

卷积层的输入是图像时，一次扫描会扫描所有通道的值再加和成一张特征图。

当卷积层的输入是上层的特征图时，特征图会被当作“通道”对待，一次扫描会扫描所有输入的特征图，加和成新的feature map。

无论在哪一层，生成的feature map 的数量都等于这一层的扫描次数，也就是等于out_channels的值。下一层卷积的in_channels就等于上一层卷积的out_channels。

---

这其实于DNN中的线性层很相似，在线性层中，下一个线性层输入的数目就等于上一个线性层的输出的数目。我们来看一下具体的卷积层的代码：

In [4]:
import torch
from torch import nn

# 假设一组数据
data = torch.ones(size= (10, 3, 28, 28)) # （samples, channels, height, width）

conv1 = nn.Conv2d(
                  in_channels= 3 # 输入信号的通道数为3
                  , out_channels= 6 # 卷积产生的通道数为6
                  , kernel_size= 3 # 卷积核的尺寸是3*3
                  )
conv2 = nn.Conv2d(
                  in_channels= 6
                  ,out_channels= 4
                  ,kernel_size= 3)

# nn.Conv2d(6, 4, 3) # in_channels=6, out_channels=4, kernel_size=3

In [5]:
conv1(data).shape # torch.Size([10, 6, 26, 26])

torch.Size([10, 6, 26, 26])

In [6]:
conv2(conv1(data)).shape # torch.Size([10, 4, 24, 24])

torch.Size([10, 4, 24, 24])

掌握卷积层的结构，对于构筑卷积神经网络非常重要。

在DNN中，每一层权重$w$都带有偏差bias，我们可以决定是否对神经网络加入偏差，在卷积中也是一样的。在这里我们的权重就是卷积核，因此每个卷积层中都可以加入偏差，偏差的数量于扫描的数量一致。当我们得到features map后，如果有偏差的存在，我们会将偏差加到feature_map的每个元素中，于矩阵+常数的计算方法一致。